In [1]:
import h5py
import os
import json
import shutil
from extratorFeatures import *

In [2]:
PROGRESS_FILE = "Datasets/progressDataset.json"
ANNOTATIONS_FOLDER = "Anotacoes_Texto/"
INCOMPLETE_SUFFIX = "_incompleto.txt"
COMPLETE_SUFFIX = "_completo.txt"
NUM_FEATURES = 15

def gerar_nome_base_anotacao(caminho_imagem):
    """
    Gera um nome de arquivo base único para a anotação, incorporando a classe (pasta pai).
    Exemplo: 'Frames2Treinamento/Fire/frame_00003.jpg' -> 'Fire_frame_00003'
    """
    nome_base_imagem = os.path.splitext(os.path.basename(caminho_imagem))[0]
    nome_classe = os.path.basename(os.path.dirname(caminho_imagem))
    return f"{nome_classe}_{nome_base_imagem}"

def salvar_progresso_imagem(caminho_imagem):
    """Salva o caminho da última imagem em que o trabalho foi iniciado."""
    progresso = {'ultimo_caminho_imagem': caminho_imagem}
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progresso, f)

def carregar_progresso_imagem():
    """Carrega o caminho da última imagem em que o trabalho foi iniciado."""
    if not os.path.exists(PROGRESS_FILE):
        return None
    with open(PROGRESS_FILE, 'r') as f:
        try:
            progresso = json.load(f)
            return progresso.get('ultimo_caminho_imagem')
        except (json.JSONDecodeError, FileNotFoundError):
            return None

def salvar_anotacao_texto(caminho_imagem, indice_contorno, classe):
    """Salva a anotação de um contorno em um arquivo de texto específico e único da imagem."""
    nome_base_anotacao = gerar_nome_base_anotacao(caminho_imagem)
    nome_arquivo_anotacao = nome_base_anotacao + INCOMPLETE_SUFFIX
    caminho_arquivo_anotacao = os.path.join(ANNOTATIONS_FOLDER, nome_arquivo_anotacao)

    with open(caminho_arquivo_anotacao, 'a') as f:
        f.write(f"{indice_contorno},{classe}\n")


def carregar_anotacoes_anteriores(caminho_imagem):
    """Carrega os índices dos contornos já anotados para uma imagem específica."""
    nome_base_anotacao = gerar_nome_base_anotacao(caminho_imagem)
    nome_incompleto = nome_base_anotacao + INCOMPLETE_SUFFIX
    nome_completo = nome_base_anotacao + COMPLETE_SUFFIX
    caminho_incompleto = os.path.join(ANNOTATIONS_FOLDER, nome_incompleto)
    caminho_completo = os.path.join(ANNOTATIONS_FOLDER, nome_completo)

    caminho_arquivo_anotacao = None
    if os.path.exists(caminho_incompleto):
        caminho_arquivo_anotacao = caminho_incompleto
    elif os.path.exists(caminho_completo):
        caminho_arquivo_anotacao = caminho_completo

    indices_anotados = set()
    if caminho_arquivo_anotacao:
        with open(caminho_arquivo_anotacao, 'r') as f:
            for linha in f:
                try:
                    indice = int(linha.strip().split(',')[0])
                    indices_anotados.add(indice)
                except (ValueError, IndexError):
                    continue
    return indices_anotados

def marcar_anotacao_como_completa(caminho_imagem):
    """Renomeia o arquivo de anotação para indicar que todos os contornos foram processados."""
    nome_base_anotacao = gerar_nome_base_anotacao(caminho_imagem)
    nome_incompleto = nome_base_anotacao + INCOMPLETE_SUFFIX
    caminho_incompleto = os.path.join(ANNOTATIONS_FOLDER, nome_incompleto)

    if os.path.exists(caminho_incompleto):
        nome_completo = nome_base_anotacao + COMPLETE_SUFFIX
        caminho_completo = os.path.join(ANNOTATIONS_FOLDER, nome_completo)

        if os.path.exists(caminho_completo):
            os.remove(caminho_completo)

        os.rename(caminho_incompleto, caminho_completo)
        print(f"  Anotação para '{os.path.basename(caminho_imagem)}' marcada como completa.")


def encontrar_imagens_incompletas():
    """Verifica a pasta de anotações e retorna uma lista de imagens com anotações incompletas."""
    imagens_incompletas = []
    if not os.path.exists(ANNOTATIONS_FOLDER):
        return imagens_incompletas

    caminho_classe_1 = 'Frames2Treinamento/Fire/'
    caminho_classe_0 = 'Frames2Treinamento/Normal/'

    mapa_imagens_unicas = {}
    if os.path.exists(caminho_classe_1):
        for f in os.listdir(caminho_classe_1):
            nome_base = os.path.splitext(f)[0]
            chave = f"Fire_{nome_base}"
            mapa_imagens_unicas[chave] = os.path.join(caminho_classe_1, f)
    if os.path.exists(caminho_classe_0):
        for f in os.listdir(caminho_classe_0):
            nome_base = os.path.splitext(f)[0]
            chave = f"Normal_{nome_base}"
            mapa_imagens_unicas[chave] = os.path.join(caminho_classe_0, f)

    for f_name in os.listdir(ANNOTATIONS_FOLDER):
        if f_name.endswith(INCOMPLETE_SUFFIX):
            chave_anotacao = f_name.replace(INCOMPLETE_SUFFIX, '')
            if chave_anotacao in mapa_imagens_unicas:
                imagens_incompletas.append(mapa_imagens_unicas[chave_anotacao])
    return imagens_incompletas


def resetar_progresso(dataset_name='Datasets/dataset_anotado.h5'):
    """Apaga arquivos de progresso, anotações e o dataset para recomeçar do zero."""
    print("--- RESETANDO PROGRESSO ---")
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
        print(f"Arquivo de progresso '{PROGRESS_FILE}' removido.")
    if os.path.exists(ANNOTATIONS_FOLDER):
        shutil.rmtree(ANNOTATIONS_FOLDER)
        print(f"Pasta de anotações '{ANNOTATIONS_FOLDER}' removida.")
    if os.path.exists(dataset_name):
        os.remove(dataset_name)
        print(f"Dataset HDF5 '{dataset_name}' removido.")
    print("\nProgresso de anotação resetado. Comece do início na próxima execução.")


def anotar_dataset_interativo(reset=False):
    """
    Processa imagens de forma interativa para anotação, salvando os resultados em arquivos de texto.
    Prioritiza a finalização de imagens marcadas como incompletas e permite voltar para corrigir a anotação anterior.
    """
    if reset:
        resetar_progresso()
        return

    os.makedirs(ANNOTATIONS_FOLDER, exist_ok=True)
    os.makedirs(os.path.dirname(PROGRESS_FILE), exist_ok=True)

    caminho_classe_1 = 'Frames2Treinamento/Fire/'
    caminho_classe_0 = 'Frames2Treinamento/Normal/'

    lista_classe_1 = [(os.path.join(caminho_classe_1, f), 'Fire') for f in sorted(os.listdir(caminho_classe_1))]
    lista_classe_0 = [(os.path.join(caminho_classe_0, f), 'Normal') for f in sorted(os.listdir(caminho_classe_0))]
    lista_completa = lista_classe_1 + lista_classe_0

    imagens_incompletas = encontrar_imagens_incompletas()
    if imagens_incompletas:
        print("--- ENCONTRADAS IMAGENS COM ANOTAÇÕES INCOMPLETAS ---")
        for img_path in imagens_incompletas:
            print(f"  - {os.path.basename(img_path)}")
        print("Finalize estas imagens primeiro.")

        lista_prioridade = [(path, 'Fire' if 'Fire' in path else 'Normal') for path in imagens_incompletas]
        caminhos_incompletos_set = set(imagens_incompletas)
        lista_restante = [item for item in lista_completa if item[0] not in caminhos_incompletos_set]
        lista_completa = lista_prioridade + lista_restante

    ultimo_caminho = carregar_progresso_imagem()
    pular_ate_encontrar = ultimo_caminho is not None and not imagens_incompletas

    print("\n--- INICIANDO ANOTAÇÃO INTERATIVA ---")
    print("  - 's': FUMAÇA (1) | 'n': NÃO FUMAÇA (0) | 'b': VOLTAR ANTERIOR")
    print("  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno")
    print(f"Pasta de anotações: {ANNOTATIONS_FOLDER}")
    if pular_ate_encontrar:
        print(f"Continuando a partir da imagem: {os.path.basename(ultimo_caminho)}")
    print("-" * 50)

    for i, (caminho_img, tipo_pasta) in enumerate(lista_completa):
        if pular_ate_encontrar and caminho_img != ultimo_caminho:
            continue
        elif pular_ate_encontrar and caminho_img == ultimo_caminho:
            pular_ate_encontrar = False
            print(f"\nRetomando anotação na imagem: {caminho_img}")
        else:
            print(f"\nProcessando nova imagem {i+1}/{len(lista_completa)}: {caminho_img}")

        img = cv2.imread(caminho_img)
        if img is None: continue

        contornos, _ = extrair_contornos(img)
        if not contornos:
            marcar_anotacao_como_completa(caminho_img)
            continue

        anotacoes_feitas = carregar_anotacoes_anteriores(caminho_img)
        salvar_progresso_imagem(caminho_img)

        quit_pressed = False
        novas_anotacoes_da_imagem = []

        j = 0
        while j < len(contornos):
            if j in anotacoes_feitas:
                j += 1
                continue

            contorno = contornos[j]
            img_display = img.copy()
            cv2.drawContours(img_display, contornos, -1, (0, 255, 0), 1)
            cv2.drawContours(img_display, [contorno], -1, (0, 255, 255), 3)

            try:
                cv2.namedWindow('Anotador - (s/n/i/q/b)', cv2.WINDOW_MAXIMIZED)
            except AttributeError:
                cv2.namedWindow('Anotador - (s/n/i/q/b)', cv2.WINDOW_NORMAL)

            cv2.imshow('Anotador - (s/n/i/q/b)', img_display)
            key = cv2.waitKey(0) & 0xFF

            if key == ord('q'):
                print("\n'q' pressionado. Saindo da imagem atual...")
                quit_pressed = True
                break

            elif key == ord('b'):
                if novas_anotacoes_da_imagem:
                    indice_removido, _ = novas_anotacoes_da_imagem.pop()
                    print(f"  Anotação para o contorno {indice_removido + 1} removida. Voltando...")
                    j -= 1
                else:
                    print("  Nenhuma anotação nesta sessão para voltar. Tente o contorno anterior.")
                    if j > 0:
                        j -= 1
                continue

            elif key == ord('s'):
                novas_anotacoes_da_imagem.append((j, 1))
                print(f"  Contorno {j+1}/{len(contornos)}: Anotado como FUMAÇA (1)")
                j += 1

            elif key == ord('n'):
                novas_anotacoes_da_imagem.append((j, 0))
                print(f"  Contorno {j+1}/{len(contornos)}: Anotado como NÃO FUMAÇA (0)")
                j += 1

            elif key == ord('i'):
                print(f"  Contorno {j+1}/{len(contornos)}: Ignorado.")
                j += 1

            else:
                print(f"  Contorno {j+1}/{len(contornos)}: Tecla inválida, ignorado.")
                j += 1

        if novas_anotacoes_da_imagem:
            print(f"  Salvando {len(novas_anotacoes_da_imagem)} nova(s) anotação(ões) no arquivo de texto...")
            for indice_contorno, classe in novas_anotacoes_da_imagem:
                salvar_anotacao_texto(caminho_img, indice_contorno, classe)

        cv2.destroyAllWindows()

        total_anotado_final = len(carregar_anotacoes_anteriores(caminho_img))
        if not quit_pressed and total_anotado_final == len(contornos):
            marcar_anotacao_como_completa(caminho_img)

        if quit_pressed:
            break

    print("\nSessão de anotação concluída.")


def criar_dataset_h5(dataset_name='Datasets/dataset_features.h5', reset_h5=False):
    """
    Cria ou atualiza um dataset HDF5 a partir dos arquivos de anotação de texto.
    Processa apenas os arquivos marcados como 'completo'.
    """
    if reset_h5 and os.path.exists(dataset_name):
        os.remove(dataset_name)
        print(f"Dataset HDF5 '{dataset_name}' existente foi removido.")

    print("\n--- INICIANDO CRIAÇÃO DO DATASET HDF5 ---")

    features_list = []
    labels_list = []

    caminho_classe_1 = 'Frames2Treinamento/Fire/'
    caminho_classe_0 = 'Frames2Treinamento/Normal/'

    mapa_imagens_unicas = {}
    if os.path.exists(caminho_classe_1):
        for f in os.listdir(caminho_classe_1):
            nome_base = os.path.splitext(f)[0]
            chave = f"Fire_{nome_base}"
            mapa_imagens_unicas[chave] = os.path.join(caminho_classe_1, f)
    if os.path.exists(caminho_classe_0):
        for f in os.listdir(caminho_classe_0):
            nome_base = os.path.splitext(f)[0]
            chave = f"Normal_{nome_base}"
            mapa_imagens_unicas[chave] = os.path.join(caminho_classe_0, f)

    arquivos_anotacao = [f for f in os.listdir(ANNOTATIONS_FOLDER) if f.endswith(COMPLETE_SUFFIX)]

    if not arquivos_anotacao:
        print("Nenhum arquivo de anotação completo encontrado para processar.")
        return

    print(f"Encontrados {len(arquivos_anotacao)} arquivos de anotação completos.")

    for nome_arquivo_anotacao in arquivos_anotacao:
        # 'Fire_frame_00003_completo.txt' -> 'Fire_frame_00003'
        chave_anotacao = nome_arquivo_anotacao.replace(COMPLETE_SUFFIX, '')
        caminho_img = mapa_imagens_unicas.get(chave_anotacao)

        if not caminho_img or not os.path.exists(caminho_img):
            print(f"Aviso: Imagem original para '{nome_arquivo_anotacao}' não encontrada. Pulando.")
            continue

        img = cv2.imread(caminho_img)
        if img is None:
            print(f"Aviso: Não foi possível ler a imagem '{caminho_img}'. Pulando.")
            continue

        contornos, img_hsv = extrair_contornos(img)

        caminho_arquivo_anotacao = os.path.join(ANNOTATIONS_FOLDER, nome_arquivo_anotacao)
        with open(caminho_arquivo_anotacao, 'r') as f:
            for linha in f:
                try:
                    indice_contorno, classe = map(int, linha.strip().split(','))
                    if 0 <= indice_contorno < len(contornos):
                        features = extrair_features(img_hsv, contornos[indice_contorno])
                        if features is not None and features.shape[0] == NUM_FEATURES:
                            features_list.append(features)
                            labels_list.append(classe)
                        else:
                            nome_base_img = os.path.basename(caminho_img)
                            print(f"  -> Aviso: Falha ao extrair features para contorno {indice_contorno} da imagem {nome_base_img}.")
                except (ValueError, IndexError):
                    continue

    if not features_list:
        print("\nNenhuma amostra válida foi extraída das anotações.")
        return

    X_data = np.array(features_list, dtype=np.float32)
    y_data = np.array(labels_list, dtype=np.int8)

    os.makedirs(os.path.dirname(dataset_name), exist_ok=True)
    with h5py.File(dataset_name, 'w') as hf:
        print(f"\nCriando novo dataset consolidado com {len(X_data)} amostras...")
        hf.create_dataset('features', data=X_data, compression="gzip", chunks=True, maxshape=(None, NUM_FEATURES))
        hf.create_dataset('labels', data=y_data, compression="gzip", chunks=True, maxshape=(None,))

    print("Operação concluída com sucesso!")
    print(f"Dataset salvo em: {dataset_name}")
    print(f"Dimensões totais: features={X_data.shape}, labels={y_data.shape}")

In [8]:
anotar_dataset_interativo(reset = False)


--- INICIANDO ANOTAÇÃO INTERATIVA ---
  - 's': FUMAÇA (1) | 'n': NÃO FUMAÇA (0) | 'b': VOLTAR ANTERIOR
  - 'q': SAIR e SALVAR | 'i': IGNORAR contorno
Pasta de anotações: Anotacoes_Texto/
Continuando a partir da imagem: frame_00267.jpg
--------------------------------------------------

Retomando anotação na imagem: Frames2Treinamento/Fire/frame_00267.jpg
  Contorno 1/84: Anotado como NÃO FUMAÇA (0)
  Contorno 2/84: Anotado como NÃO FUMAÇA (0)
  Contorno 3/84: Anotado como NÃO FUMAÇA (0)
  Contorno 4/84: Anotado como NÃO FUMAÇA (0)
  Contorno 5/84: Anotado como NÃO FUMAÇA (0)
  Contorno 6/84: Anotado como NÃO FUMAÇA (0)
  Contorno 7/84: Anotado como NÃO FUMAÇA (0)
  Contorno 8/84: Anotado como NÃO FUMAÇA (0)
  Contorno 9/84: Anotado como NÃO FUMAÇA (0)
  Contorno 10/84: Anotado como NÃO FUMAÇA (0)
  Contorno 11/84: Anotado como NÃO FUMAÇA (0)
  Contorno 12/84: Anotado como NÃO FUMAÇA (0)
  Contorno 13/84: Anotado como NÃO FUMAÇA (0)
  Contorno 14/84: Anotado como NÃO FUMAÇA (0)
  Conto

In [12]:
scriar_dataset_h5(reset_h5 = True)

Dataset HDF5 'Datasets/dataset_features.h5' existente foi removido.

--- INICIANDO CRIAÇÃO DO DATASET HDF5 ---
Encontrados 2 arquivos de anotação completos.

Criando novo dataset consolidado com 52 amostras...
Operação concluída com sucesso!
Dataset salvo em: Datasets/dataset_features.h5
Dimensões totais: features=(52, 15), labels=(52,)
